In [50]:
# Step 1 - Install and Import Packages
!pip install -q pandas matplotlib seaborn streamlit pyngrok plotly
import os
import json
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import plotly.express as px
from pyngrok import ngrok

In [51]:
# Step 2 - Clone the Pulse GitHub Repository
!rm -rf pulse
!git clone https://github.com/PhonePe/pulse.git

Cloning into 'pulse'...
remote: Enumerating objects: 17904, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 17904 (delta 19), reused 17 (delta 17), pack-reused 17855 (from 2)
Receiving objects: 100% (17904/17904), 26.13 MiB | 8.07 MiB/s, done.
Resolving deltas: 100% (8723/8723), done.
Updating files: 100% (9029/9029), done.


In [52]:
# Step 3 - Create SQLite Database and Tables
conn = sqlite3.connect("phonepe.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS aggregated_transaction (
    state TEXT, year INTEGER, quarter INTEGER,
    transaction_type TEXT, count INTEGER, amount REAL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS aggregated_user_summary (
    state TEXT, year INTEGER, quarter INTEGER,
    registered_users INTEGER, app_opens INTEGER
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS aggregated_user_by_device (
    state TEXT, year INTEGER, quarter INTEGER,
    brand TEXT, user_count INTEGER, percentage REAL
)
""")

conn.commit()
print("✅ All tables created.")

✅ All tables created.


In [53]:
# Step 4 - Extract and Load Data

def extract_transaction_data():
    root_dir = "pulse/data/aggregated/transaction/country/india"
    for year in os.listdir(root_dir):
        year_path = os.path.join(root_dir, year)
        if not os.path.isdir(year_path):
            continue
        for file in os.listdir(year_path):
            try:
                quarter = int(file.replace(".json", ""))
            except ValueError:
                continue
            file_path = os.path.join(year_path, file)
            with open(file_path, 'r') as f:
                js = json.load(f)
                if js['data'].get('transactionData'):
                    for tx in js['data']['transactionData']:
                        for item in tx['paymentInstruments']:
                            cursor.execute("""
                            INSERT INTO aggregated_transaction VALUES (?, ?, ?, ?, ?, ?)
                            """, ("India", int(year), quarter, tx['name'], item['count'], item['amount']))
    conn.commit()

def extract_user_data():
    root_dir = "pulse/data/aggregated/user/country/india"
    for year in os.listdir(root_dir):
        year_path = os.path.join(root_dir, year)
        if not os.path.isdir(year_path):
            continue
        for file in os.listdir(year_path):
            try:
                quarter = int(file.replace(".json", ""))
            except ValueError:
                continue
            file_path = os.path.join(year_path, file)
            with open(file_path, 'r') as f:
                js = json.load(f).get("data", {})
                summary = js.get("aggregated", {})
                cursor.execute("INSERT INTO aggregated_user_summary VALUES (?, ?, ?, ?, ?)",
                               ("India", int(year), quarter, summary.get("registeredUsers", 0), summary.get("appOpens", 0)))
                for device in js.get("usersByDevice", []) or []:
                    cursor.execute("INSERT INTO aggregated_user_by_device VALUES (?, ?, ?, ?, ?, ?)",
                                   ("India", int(year), quarter, device.get("brand", "Unknown"), device.get("count", 0), device.get("percentage", 0)))
    conn.commit()

extract_transaction_data()
extract_user_data()
print("✅ Data extraction complete.")

✅ Data extraction complete.


In [56]:
# Step 5 - Visualizations (8 Key Findings)

# Top Transaction Types
query1 = """
SELECT transaction_type, SUM(amount) as total_amt
FROM aggregated_transaction
GROUP BY transaction_type
ORDER BY total_amt DESC LIMIT 5
"""
df1 = pd.read_sql(query1, conn)
fig1 = px.bar(df1, x="total_amt", y="transaction_type", orientation='h', color="transaction_type")
fig1.show()

# Transaction Volume Over Time
query2 = """
SELECT year, quarter, SUM(count) as txn_count
FROM aggregated_transaction
GROUP BY year, quarter
ORDER BY year, quarter
"""
df2 = pd.read_sql(query2, conn)
df2['period'] = df2['year'].astype(str) + ' Q' + df2['quarter'].astype(str)
fig2 = px.line(df2, x="period", y="txn_count", markers=True)
fig2.show()

# App Opens Per Quarter
query3 = """
SELECT year, quarter, SUM(app_opens) as total_opens
FROM aggregated_user_summary
GROUP BY year, quarter
ORDER BY year, quarter
"""
df3 = pd.read_sql(query3, conn)
df3['period'] = df3['year'].astype(str) + ' Q' + df3['quarter'].astype(str)
fig3 = px.area(df3, x="period", y="total_opens")
fig3.show()

# Registered Users Over Time
query4 = """
SELECT year, quarter, SUM(registered_users) as users
FROM aggregated_user_summary
GROUP BY year, quarter
ORDER BY year, quarter
"""
df4 = pd.read_sql(query4, conn)
df4['period'] = df4['year'].astype(str) + ' Q' + df4['quarter'].astype(str)
fig4 = px.line(df4, x="period", y="users")
fig4.show()

# Top Devices Used
query5 = """
SELECT brand, SUM(user_count) as total_users
FROM aggregated_user_by_device
GROUP BY brand
ORDER BY total_users DESC LIMIT 5
"""
df5 = pd.read_sql(query5, conn)
fig5 = px.pie(df5, names="brand", values="total_users")
fig5.show()

# Average Transaction Amount Over Time
query6 = """
SELECT year, quarter, SUM(amount)*1.0/SUM(count) as avg_txn_amt
FROM aggregated_transaction
GROUP BY year, quarter
ORDER BY year, quarter
"""
df6 = pd.read_sql(query6, conn)
df6['period'] = df6['year'].astype(str) + ' Q' + df6['quarter'].astype(str)
fig6 = px.bar(df6, x="period", y="avg_txn_amt")
fig6.show()

# Most Active Year
query7 = "SELECT year, SUM(count) as txn_count FROM aggregated_transaction GROUP BY year ORDER BY txn_count DESC"
df7 = pd.read_sql(query7, conn)
fig7 = px.bar(df7, x="year", y="txn_count")
fig7.show()

# Most Popular Quarters for App Opens
query8 = """
SELECT quarter, SUM(app_opens) as total_opens FROM aggregated_user_summary
GROUP BY quarter ORDER BY total_opens DESC
"""
df8 = pd.read_sql(query8, conn)
fig8 = px.funnel(df8, x="total_opens", y="quarter")
fig8.show()

In [57]:
# Step 6 - Streamlit App for PhonePe Transaction Insights

import streamlit as st
import pandas as pd
import sqlite3
import plotly.express as px

# Connect to SQLite DB
conn = sqlite3.connect("phonepe.db")

st.set_page_config(page_title="📊 PhonePe Insights", layout="wide")
st.title("📊 PhonePe Transaction Insights Dashboard")
st.subheader("8 Key Findings with Unique Visualizations")

# Top Transaction Types
query1 = """
SELECT transaction_type, SUM(amount) as total_amt
FROM aggregated_transaction
WHERE amount > 0
GROUP BY transaction_type
ORDER BY total_amt DESC LIMIT 5
"""
df1 = pd.read_sql(query1, conn)
fig1 = px.bar(df1, x='transaction_type', y='total_amt',
              title="Top Transaction Types",
              color='transaction_type',
              labels={'total_amt': 'Total Amount'})
fig1.update_layout(yaxis_range=[0, df1['total_amt'].max() * 1.1], yaxis_tickformat=",", xaxis_tickangle=-45)
st.plotly_chart(fig1, use_container_width=True)

# Transaction Volume Over Time
query2 = """
SELECT year, quarter, SUM(count) as txn_count
FROM aggregated_transaction
GROUP BY year, quarter
ORDER BY year, quarter
"""
df2 = pd.read_sql(query2, conn)
df2['period'] = df2['year'].astype(str) + ' Q' + df2['quarter'].astype(str)
fig2 = px.line(df2, x='period', y='txn_count', title="Transaction Volume Over Time", markers=True)
fig2.update_layout(xaxis_tickangle=-45)
st.plotly_chart(fig2, use_container_width=True)

# App Opens Per Quarter
query3 = """
SELECT year, quarter, SUM(app_opens) as total_opens
FROM aggregated_user_summary
GROUP BY year, quarter
ORDER BY year, quarter
"""
df3 = pd.read_sql(query3, conn)
df3['period'] = df3['year'].astype(str) + ' Q' + df3['quarter'].astype(str)
fig3 = px.area(df3, x='period', y='total_opens', title="App Opens Per Quarter")
st.plotly_chart(fig3, use_container_width=True)

# Registered Users Over Time
query4 = """
SELECT year, quarter, SUM(registered_users) as users
FROM aggregated_user_summary
GROUP BY year, quarter
ORDER BY year, quarter
"""
df4 = pd.read_sql(query4, conn)
df4['period'] = df4['year'].astype(str) + ' Q' + df4['quarter'].astype(str)
fig4 = px.line(df4, x='period', y='users', title="Registered Users Over Time")
st.plotly_chart(fig4, use_container_width=True)

# Top Devices Used
query5 = """
SELECT brand, SUM(user_count) as total_users
FROM aggregated_user_by_device
GROUP BY brand
ORDER BY total_users DESC LIMIT 5
"""
df5 = pd.read_sql(query5, conn)
fig5 = px.pie(df5, names='brand', values='total_users', title="Top 5 Devices Used")
st.plotly_chart(fig5, use_container_width=True)

# Average Transaction Amount Over Time
query6 = """
SELECT year, quarter, SUM(amount)/SUM(count) as avg_amount
FROM aggregated_transaction
GROUP BY year, quarter
ORDER BY year, quarter
"""
df6 = pd.read_sql(query6, conn)
df6['period'] = df6['year'].astype(str) + ' Q' + df6['quarter'].astype(str)
fig6 = px.bar(df6, x='period', y='avg_amount', title="Average Transaction Amount Over Time")
fig6.update_layout(xaxis_tickangle=-45)
st.plotly_chart(fig6, use_container_width=True)

# Most Active Transaction Years
query7 = """
SELECT year, SUM(count) as txn_count FROM aggregated_transaction
GROUP BY year ORDER BY txn_count DESC
"""
df7 = pd.read_sql(query7, conn)
fig7 = px.bar(df7, x='year', y='txn_count', title="Most Active Transaction Years")
st.plotly_chart(fig7, use_container_width=True)

# Most Popular Quarters for App Opens
query8 = """
SELECT quarter, SUM(app_opens) as total_opens FROM aggregated_user_summary
GROUP BY quarter ORDER BY total_opens DESC
"""
df8 = pd.read_sql(query8, conn)
fig8 = px.funnel(df8, x='total_opens', y='quarter', title="Most Popular Quarters for App Opens")
st.plotly_chart(fig8, use_container_width=True)
st.success("📊 Dashboard loaded successfully with 8 Key Findings!")

2025-06-25 17:33:00.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 17:33:00.077 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 17:33:00.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 17:33:00.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 17:33:00.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 17:33:00.083 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 17:33:00.085 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 17:33:00.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [58]:
# Step 7 - Run the Streamlit
from pyngrok import ngrok
ngrok.set_auth_token("2z0Oqv0tD166fELGCHwV2gLZwq1_2G2zUQRSs6C27k9vdzxwq")
!streamlit run app.py &> /content/logs.txt &
public_url = ngrok.connect(8501)
print("🚀 Streamlit is running at:", public_url)

🚀 Streamlit is running at: NgrokTunnel: "https://acaf-34-28-251-35.ngrok-free.app" -> "http://localhost:8501"
